# Ejercicio práctico de TensorFlow

En este ejercicio pondremos en práctica los elementos más básicos de TensorFlow. Específicamente, usaremos `tf.placeholder`, `tf.Variable` y `tf.constant`, así como `tf.Graph` y `tf.Session`.

En este ejercicio vamos a calcular el costo de comer en un restaurante. En una típica salida a un restaruante:

*   Seleccionamos los platos que vamos a consumir
*   Mientras dura el tiempo en el restaurante, podríamos repetir alguno de los productos consumidos, por lo que es importante llevar la cuenta de cuántas veces consumimos cada producto.
*   Al costo de lo que consumimos debemos añadirle el iva, el cual es el 19% de lo que consumimos.
*   En caso de decidir pagar propina, debemos sumarlo al costo de las comidas y bebidas que consumimos. 
*   Finalmente, es problable que hayamos ido a comer con varias personas, por lo que hace falta dividir la cuenta entre el número de personas.



In [0]:
import numpy as np
import tensorflow as tf

Para este ejercicio, vamos a crear primero un arreglo que nos guarde los nombres de los productos del restaurante, de esta forma es evidente que cada producto tiene un índice

In [25]:
productos = np.array(['cerveza', 'torta de zanahoria', 'torta de banano',
                      'torta bavariana', 'pizza napolitana',
                      'pizza cuatro quesos', 'pizza vegetales',
                      'sopa de tomate','humus'])
productos

array(['cerveza', 'torta de zanahoria', 'torta de banano',
       'torta bavariana', 'pizza napolitana', 'pizza cuatro quesos',
       'pizza vegetales', 'sopa de tomate', 'humus'], dtype='<U19')

También, vamos a crear un tensor constante en el que pondremos el precio de cada producto.

In [0]:
precios = tf.constant(
    [5000, 8000, 8500, 9500, 20000, 28000, 24000, 10000, 12000],
    dtype=tf.float32, shape=[len(productos)], name="precios"
)

**Ejercicio 1:** crea un placeholder para ingresar los datos de una compra. Este placeholder debe admitir una cantidad de valores igual al número productos, para posteriormente ser llenado con enteros que representen la cantidad de veces que se ordenó cada producto.

In [0]:
compra = tf.placeholder(dtype=tf.float32, shape=[len(productos)], name="compra")

**Ejercicio 2:** multiplica el tensor de compras con el tensor de precios elemento por elemento, para obtener un precio por producto que incluya la cantidad de veces que se ordenó un producto. Usa [`tf.multiply`](https://www.tensorflow.org/api_docs/python/tf/multiply)

In [0]:
costo_por_producto = tf.multiply(precios, compra, name="costo_por_producto")

**Ejercicio 3:** suma el valor de `costo_por_producto `, de tal forma que puedas obtener el costo base de la compra. Utiliza [`tf.reduce_sum`](https://www.tensorflow.org/api_docs/python/tf/reduce_sum).

In [0]:
costo_base = tf.reduce_sum(costo_por_producto, name="costo_base")

**Ejercicio 4:** calcula el iva de la compra que acabas de hacer, para esto crea una constante que te guarde el porcentaje de la compra que debes sumar al costo base (0.19) y luego multiplica el costo base por este valor para obtener el valor que debes sumar a la cuenta. No te preocupes si te sale el siguiente error: `TypeError: Input 'y' of 'Mul' Op has type int32 that does not match type float32 of argument 'x'.`

In [0]:
porciento_iva = tf.constant(0.19, name='porciento_iva', dtype=tf.float32)
costo_iva = tf.multiply(porciento_iva, costo_base, name='costo_iva')

**Nota:** si no te salió el mensaje de error, puedes ignorar esta celda.


Podemos ver que TensorFlow nos dice que no se puede multiplicar un `int32` con un `float32`, por lo que debemos convertir el tipo de dato de la variable `costo_base`

**Ejercicio 4.1:** redefine la variable `costo_base` de tal forma que el tipo de variable sea `float32`, utiliza [`tf.cast`](https://www.tensorflow.org/api_docs/python/tf/cast). Posteriormente, veulve a calcular el iva.

In [0]:
costo_base = tf.cast(
    tf.reduce_sum(costo_por_producto, name="costo_base"), dtype=tf.float32
)

porciento_iva = tf.constant(0.19, name='porciento_iva')
costo_iva = tf.multiply(
    porciento_iva,
    costo_base,
    name='costo_iva'
)

Ahora falta incluir la propina, dado que no es algo obligatorio, vamos a tratar la propina como un `placeholder` en el cual simplemente se ingresará el valor de la propina que el cliente quiera dar. 

**Ejercicio 5:**  crea un [`tf.placeholder_with_default`](https://www.tensorflow.org/api_docs/python/tf/placeholder_with_default) utilizando `0` como el valor por defecto.

In [0]:
propina = tf.placeholder_with_default(0.0, shape=None, name="propina")

Ya tenemos todo lo necesario para calcular el costo total.

**Ejercicio 6:** calcula el costo total de la salida al restaurante, el cual es la suma del costo base, el costo por iva y la propina.

In [0]:
costo_total = costo_base + costo_iva + propina

Finalmente, es probable que queramos dividir la cuenta entre algún número de personas.

**Ejercicio 7:** crea un `tf.placeholder_with_default` que guarde el número de personas en los que se va a dividir la cuenta. El valor por defecto debe ser `1`.

In [0]:
n_personas = tf.placeholder_with_default(1.0, shape=None, name='n_personas')
costo_por_persona = costo_total/n_personas

Ya hemos terminado de definir el grafo, ahora solo queda ejecutarlo.

**Ejercicio 8:** completa la siguiente celda de tal forma que imprima el costo total y el costo por persona, asumiendo que la cuenta sólo se pagará por una persona y no habrá propina. La compra será la siguiente:

*   5 cervezas
*   1 torta de zanahoria
*   1 torta de banano
*   3 pizzas napolitanas
*   1 sopa de tomate
*   12 platos de humus

In [36]:
with tf.Session() as sess:
    
    ct, cp = sess.run(
        [costo_total, costo_por_persona],
        feed_dict={
            compra:[5, 1, 1, 0, 3, 0, 0, 1, 12]
        }
    )
    
    
    print("The total cost is: {}".format(ct))
    print("Each person should pay: {}".format(cp))

The total cost is: 304045.0
Each person should pay: 304045.0


**Ejercicio 9:** repite el ejercicio anterior asumiendo una propina de $5000 y que la cuenta se divide en 4 personas

In [37]:
with tf.Session() as sess:
    
    ct, cp = sess.run(
        [costo_total, costo_por_persona],
        feed_dict={
            compra: [5, 1, 1, 0, 3, 0, 0, 1, 12],
            propina: 5000,
            n_personas: 4
        }
    )
    
    
    print("The total cost is: {}".format(ct))
    print("Each person should pay: {}".format(cp))

The total cost is: 309045.0
Each person should pay: 77261.25
